In [ ]:
import numpy as np
import pandas as pd
import pickle
data = pickle.load(open('../data/60_second_data1.p','rb'))

In [ ]:
from hrvanalysis import get_time_domain_features
from joblib import Parallel,delayed
data.shape

In [ ]:
def get_data(a):
    features = []
    ecg_rr = a[:,-1]
    if len(ecg_rr[np.isnan(ecg_rr)])>40:
        return [],[],[],[],[],[]
    m = np.mean(ecg_rr[ecg_rr>0])
    s = np.std(ecg_rr[ecg_rr>0])
    ecg_rr[np.isnan(ecg_rr)] = 0
    if len(ecg_rr)<60:
        return [],[],[],[],[],[]
    y = []
    X = []
    ecg = []
    means = []
    stds = []
    quals = []
    for i in [-2]:
        ppg_rr = a[:,i]
        ppg_qual = a[:,i-4]
#         index = ppg_rr>0
#         ppg_qual = ppg_qual[index]
#         ppg_rr = ppg_rr[index]
        index = np.isnan(ppg_rr)
        index1 = ~np.isnan(ppg_rr)
        if len(ppg_rr[index1])<10:
            continue
        ppg_qual[index] = -1
        ppg_rr[index] = np.nanmean(ppg_rr)
        y.append(ppg_rr.reshape(1,60,1))
        tmp = a[:,np.array([-2,-6,1])].reshape(1,60,3)
        tmp[np.isnan(tmp)] = 0
        tmp[tmp==0] = 0
        X.append(tmp)
        means.append(m)
        stds.append(s)
        ecg.append(ecg_rr.reshape(1,60,1))
        quals.append(ppg_qual.reshape(1,60,1))
#         for j in np.linspace(0,.9,20):
#             index = ppg_qual>j
#             ppg_qual = ppg_qual[index]
#             ppg_rr = ppg_rr[index]
#             if len(ppg_rr)<10:
#                 continue
#             f = list(get_time_domain_features(ppg_rr).values())
#             f1 = list(get_time_domain_features(ecg_rr).values())
#             q = [np.percentile(ppg_qual,20),np.median(ppg_qual),len(ppg_rr)/60]
#             features.append(np.array(f1+f+q))
    return X,y,ecg,means,stds,quals

df_col = Parallel(n_jobs=10,verbose=1)(delayed(get_data)(a) for a in data if len(a[~np.isnan(a[:,-1]),-1])>20)    

In [ ]:
X,y,ecg,means,stds,quals = [],[],[],[],[],[]
for a in df_col:
    if len(a[0])==0:
        continue
    X.extend(a[0])
    y.extend(a[1])
    ecg.extend(a[2])
    means.extend(a[3])
    stds.extend(a[4])
    quals.extend(a[5])

In [ ]:
np.concatenate(y).shape

In [ ]:
X,y,ecg,means,stds,quals = np.concatenate(X),np.concatenate(y).reshape(-1,60),np.concatenate(ecg).reshape(-1,60),\
np.array(means).reshape(-1,1),np.array(stds).reshape(-1,1),np.concatenate(quals)

In [ ]:
from keras.layers import Input, LSTM, RepeatVector,Bidirectional,Multiply,multiply,Permute
from keras.layers import TimeDistributed,Dense,Flatten,Reshape,Lambda,Activation,GRU
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import numpy as np
import keras.backend as K
from keras import metrics,losses
import tensorflow as tf
# import tensorflow_probability as tfp
X_train, X_test, y_train, y_test,ecg_train, \
ecg_test,means_train,means_test,stds_train,stds_test, \
quals_train,quals_test= train_test_split(
    X, y,ecg,means,stds,quals, test_size=0.33, random_state=42)
X_train, X_val, y_train, y_val,means_train,means_val,stds_train,stds_val = train_test_split(
    X_train, y_train,means_train,stds_train, test_size=0.2, random_state=42)
print(X.shape,y.shape,y_val.shape,means_val.shape,stds_val.shape)

In [265]:
def output_of_lambda(input_shape):
    return (input_shape[0], 1)

def mean(x):
    return K.mean(x, axis=1, keepdims=True)

def output_of_lambda1(input_shape):
    return (input_shape[0], 1)

def mean1(x):
    return K.std(x, axis=1, keepdims=True)

timesteps = 60
input_dim = 3
latent_dim = 20
output_dim = 1
n = 1
inputs = Input(shape=(timesteps, input_dim))
# inputs2 = Reshape((1,1))(inputs1)
encoded = Bidirectional(GRU(60,return_sequences=True,activation='relu',go_backwards=True))(inputs)
# encoded = LSTM(output_dim,return_sequences=True,activation='sigmoid')(encoded)
att = TimeDistributed(Dense(1,activation='relu'))(encoded)

att = Flatten()(att)
att = Activation(activation="softmax")(att)
att = RepeatVector(120)(att)
att = Permute((2,1))(att)
mer = multiply([att, encoded])

encoded = TimeDistributed(Dense(10,activation='relu'))(mer)
encoded = Flatten()(encoded)
# encoded = Dense(10,activation='relu',name='sequence1')(encoded)
encoded = Dense(60,activation='relu',name='sequence')(encoded)
# encoded = Reshape((60),name='sequence')(encoded)
# encoded_std = K.std(encoded,axis=1)
decoded = Lambda(mean, output_shape=output_of_lambda,name='mean')(encoded)
decoded1 = Lambda(mean1, output_shape=output_of_lambda1,name='std')(encoded)
# decoded = LSTM(output_dim*60, return_sequences=True)(decoded)
# decoded = LSTM(output_dim*3, return_sequences=True)(decoded)
# decoded = LSTM(output_dim, return_sequences=True)(decoded)

sequence_autoencoder = Model(inputs=[inputs], outputs=[encoded,decoded,decoded1])
# encoder = Model(inputs, encoded)
losses = {
    "std":"logcosh",
    "mean": "logcosh",
    "sequence": "mae"
}
lossWeights = {"mean": 1, "sequence": 1,"std":1}
# initialize the optimizer and compile the model

sequence_autoencoder.compile(optimizer='adam',loss=losses, loss_weights=lossWeights)

sequence_autoencoder.summary()

Model: "model_23"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_37 (InputLayer)           (None, 60, 3)        0                                            
__________________________________________________________________________________________________
bidirectional_37 (Bidirectional (None, 60, 120)      23040       input_37[0][0]                   
__________________________________________________________________________________________________
time_distributed_36 (TimeDistri (None, 60, 1)        121         bidirectional_37[0][0]           
__________________________________________________________________________________________________
flatten_38 (Flatten)            (None, 60)           0           time_distributed_36[0][0]        
___________________________________________________________________________________________

In [266]:
filepath = '../models/base_LSTM.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=30)
callbacks_list = [es,checkpoint]
history = sequence_autoencoder.fit(X_train, [y_train,means_train,stds_train],
                epochs=300,
                batch_size=500,
                shuffle=True,
                validation_data=(X_val,[y_val,means_val,stds_val]),callbacks=callbacks_list)

Train on 14228 samples, validate on 3557 samples
Epoch 1/300
14228/14228 [==============================] - 6s 412us/step - loss: 1561.4482 - sequence_loss: 752.2248 - mean_loss: 772.4379 - std_loss: 35.1775 - val_loss: 1490.8868 - val_sequence_loss: 727.0180 - val_mean_loss: 741.1714 - val_std_loss: 20.2614

Epoch 00001: val_loss improved from inf to 1490.88676, saving model to ../models/base_LSTM.hdf5
Epoch 2/300
14228/14228 [==============================] - 14s 999us/step - loss: 1235.8447 - sequence_loss: 589.9869 - mean_loss: 591.7291 - std_loss: 44.0152 - val_loss: 647.9706 - val_sequence_loss: 304.4780 - val_mean_loss: 218.6539 - val_std_loss: 121.4944

Epoch 00002: val_loss improved from 1490.88676 to 647.97059, saving model to ../models/base_LSTM.hdf5
Epoch 3/300
14228/14228 [==============================] - 18s 1ms/step - loss: 437.7117 - sequence_loss: 229.9534 - mean_loss: 132.8577 - std_loss: 72.7345 - val_loss: 305.5823 - val_sequence_loss: 186.3163 - val_mean_loss: 96.

14228/14228 [==============================] - 18s 1ms/step - loss: 246.1478 - sequence_loss: 155.9138 - mean_loss: 72.4069 - std_loss: 17.7727 - val_loss: 241.0851 - val_sequence_loss: 155.9550 - val_mean_loss: 67.6521 - val_std_loss: 17.2631

Epoch 00024: val_loss improved from 241.79928 to 241.08510, saving model to ../models/base_LSTM.hdf5
Epoch 25/300
14228/14228 [==============================] - 18s 1ms/step - loss: 245.2666 - sequence_loss: 155.6236 - mean_loss: 72.0059 - std_loss: 17.7100 - val_loss: 242.3769 - val_sequence_loss: 156.5247 - val_mean_loss: 68.0660 - val_std_loss: 17.2756

Epoch 00025: val_loss did not improve from 241.08510
Epoch 26/300
14228/14228 [==============================] - 18s 1ms/step - loss: 247.4177 - sequence_loss: 156.6955 - mean_loss: 72.9023 - std_loss: 17.7799 - val_loss: 244.0180 - val_sequence_loss: 157.2073 - val_mean_loss: 68.9674 - val_std_loss: 17.3078

Epoch 00026: val_loss did not improve from 241.08510
Epoch 27/300
14228/14228 [======

14228/14228 [==============================] - 18s 1ms/step - loss: 244.2159 - sequence_loss: 155.6568 - mean_loss: 70.9856 - std_loss: 17.7007 - val_loss: 238.6580 - val_sequence_loss: 155.3829 - val_mean_loss: 66.2438 - val_std_loss: 17.0728

Epoch 00049: val_loss did not improve from 238.08510
Epoch 50/300
14228/14228 [==============================] - 18s 1ms/step - loss: 242.8174 - sequence_loss: 154.9261 - mean_loss: 70.4241 - std_loss: 17.7471 - val_loss: 238.1495 - val_sequence_loss: 155.0252 - val_mean_loss: 65.8114 - val_std_loss: 17.1705

Epoch 00050: val_loss did not improve from 238.08510
Epoch 51/300
14228/14228 [==============================] - 18s 1ms/step - loss: 242.0873 - sequence_loss: 154.7169 - mean_loss: 70.1370 - std_loss: 17.7229 - val_loss: 238.8710 - val_sequence_loss: 155.0242 - val_mean_loss: 66.3753 - val_std_loss: 17.2598

Epoch 00051: val_loss did not improve from 238.08510
Epoch 52/300
14228/14228 [==============================] - 18s 1ms/step - loss:

14228/14228 [==============================] - 18s 1ms/step - loss: 242.2664 - sequence_loss: 154.7403 - mean_loss: 70.0082 - std_loss: 17.7279 - val_loss: 238.9082 - val_sequence_loss: 155.2435 - val_mean_loss: 66.1023 - val_std_loss: 17.2921

Epoch 00075: val_loss did not improve from 236.93630
Epoch 76/300
14228/14228 [==============================] - 18s 1ms/step - loss: 241.7133 - sequence_loss: 154.3310 - mean_loss: 69.5013 - std_loss: 17.7552 - val_loss: 238.8517 - val_sequence_loss: 155.1463 - val_mean_loss: 66.4948 - val_std_loss: 17.2874

Epoch 00076: val_loss did not improve from 236.93630
Epoch 77/300
14228/14228 [==============================] - 18s 1ms/step - loss: 241.9795 - sequence_loss: 154.6412 - mean_loss: 69.4603 - std_loss: 17.7518 - val_loss: 238.5456 - val_sequence_loss: 154.7374 - val_mean_loss: 66.5819 - val_std_loss: 17.2042

Epoch 00077: val_loss did not improve from 236.93630
Epoch 78/300
14228/14228 [==============================] - 18s 1ms/step - loss:

14228/14228 [==============================] - 18s 1ms/step - loss: 236.9312 - sequence_loss: 149.5914 - mean_loss: 67.9112 - std_loss: 19.5902 - val_loss: 235.7628 - val_sequence_loss: 151.5897 - val_mean_loss: 65.3897 - val_std_loss: 18.5303

Epoch 00100: val_loss did not improve from 232.67703
Epoch 101/300
14228/14228 [==============================] - 18s 1ms/step - loss: 235.9159 - sequence_loss: 148.9803 - mean_loss: 67.7554 - std_loss: 19.0832 - val_loss: 227.9274 - val_sequence_loss: 147.4551 - val_mean_loss: 62.5682 - val_std_loss: 18.0150

Epoch 00101: val_loss improved from 232.67703 to 227.92741, saving model to ../models/base_LSTM.hdf5
Epoch 102/300
14228/14228 [==============================] - 18s 1ms/step - loss: 240.7424 - sequence_loss: 151.5777 - mean_loss: 68.3379 - std_loss: 20.5746 - val_loss: 235.5527 - val_sequence_loss: 152.3675 - val_mean_loss: 64.0438 - val_std_loss: 18.5699

Epoch 00102: val_loss did not improve from 227.92741
Epoch 103/300
14228/14228 [===

14228/14228 [==============================] - 9s 613us/step - loss: 274.6547 - sequence_loss: 168.8504 - mean_loss: 87.0742 - std_loss: 18.6609 - val_loss: 260.1796 - val_sequence_loss: 167.2428 - val_mean_loss: 74.6496 - val_std_loss: 17.9228

Epoch 00126: val_loss did not improve from 227.92741
Epoch 127/300
14228/14228 [==============================] - 9s 618us/step - loss: 258.6902 - sequence_loss: 163.3105 - mean_loss: 76.9824 - std_loss: 18.2441 - val_loss: 253.0420 - val_sequence_loss: 161.8730 - val_mean_loss: 73.0708 - val_std_loss: 17.6565

Epoch 00127: val_loss did not improve from 227.92741
Epoch 128/300
14228/14228 [==============================] - 9s 612us/step - loss: 255.4740 - sequence_loss: 162.4109 - mean_loss: 74.8945 - std_loss: 18.0756 - val_loss: 255.8814 - val_sequence_loss: 163.8774 - val_mean_loss: 74.0018 - val_std_loss: 17.7912

Epoch 00128: val_loss did not improve from 227.92741
Epoch 129/300
14228/14228 [==============================] - 9s 613us/step 

In [ ]:
from keras.models import load_model
sequence_autoencoder = load_model(filepath)

In [ ]:
y_pred1 = sequence_autoencoder.predict(X_test)
y_pred = y_pred1[0]
mean_pred = y_pred1[1]
stds_pred = y_pred1[2]

In [ ]:
y_pred1[0].shape,y_pred1[1].shape,y_pred1[2].shape

In [ ]:
%matplotlib  inline
import matplotlib.pyplot as plt
# plt.plot(y[-10])
for i,a in enumerate(y_pred[::-1][20:100]):
#     if np.sum(a)>0:
    plt.figure()
    plt.plot(a,'g')
#         plt.plot(X[i,:,:],'r')
#     plt.plot(y_test[i],'b')
    plt.plot(ecg_test[i],'r')
    plt.show()

In [ ]:
X = np.zeros((len(quals_test.reshape(-1)),4))
X[:,0] = quals_test.reshape(-1)
X[:,1] = y_pred.reshape(-1)
X[:,2] = ecg_test.reshape(-1)
X[:,3] = y_test.reshape(-1)

In [ ]:
X = X[X[:,0]>-1]
X = X[X[:,2]>0]
X = X[X[:,3]>0]

In [ ]:
l_range = np.arange(0,1,.05)
x = []
y = []
y1 = []
for l in l_range:
    index = np.where((X[:,0]>=l)&(X[:,0]<l+.05))[0]
    temp = X[index]
    x.append(str(np.round(l*100)/100)+'-'+str(np.round((l+.05)*100)/100))
    y.append(list(np.abs(temp[:,2]-temp[:,3])))
    y1.append(list(np.abs(temp[:,1]-temp[:,2])))
#     print(np.mean(np.abs(temp[:,0]-temp[:,2])),np.std(np.abs(temp[:,0]-temp[:,2])),len(index))
plt.figure(figsize=(10,8))
plt.rcParams.update({'font.size':20})
c = plt.boxplot(y,showfliers=False,positions=np.array(range(0,3*len(y),3)),notch=True)
for box in c['boxes']:
    box.set(color='red', linewidth=1)
b = plt.boxplot(y1,showfliers=False,positions=np.array(range(0,3*len(y),3))+1.5,notch=True)
for box in b['boxes']:
    box.set(color='blue', linewidth=1)
#     box.set(facecolor = 'red' )
plt.xticks(np.array(range(0,3*len(y),3)),x,rotation=60)
plt.ylabel('Absolute Difference in Milliseconds')
plt.xlabel('Range of Signal Quality')
plt.tight_layout()
plt.show()   

In [ ]:
from hrvanalysis import get_time_domain_features
x = []
y = []
z = []
q = []
s = 'std_hr'
for i in range(ecg_test.shape[0]):
    qual_min = quals_test[i].reshape(-1)
    qual_min = qual_min[qual_min>-1]
    ecg_min = ecg_test[i].reshape(-1)
    ecg_min = ecg_min[ecg_min>0]
    ecg_min= ecg_min[~np.isnan(ecg_min)]
    y_pred_min = y_pred[i].reshape(-1)
    y_pred_min = y_pred_min[~np.isnan(y_pred_min)]
    y_test_min = y_test[i].reshape(-1)
    y_test_min = y_test_min[y_test_min>0]
    y_test_min = y_test_min[~np.isnan(y_test_min)]
    
    x.append(get_time_domain_features(y_pred_min)[s])
    y.append(get_time_domain_features(ecg_min)[s])
    z.append(get_time_domain_features(y_test_min)[s])
    q.append(np.median(qual_min))
    if np.isinf(x[-1]) or np.isinf(y[-1]) or np.isinf(z[-1]) or np.isinf(q[-1]):
        x = x[:-1]
        y= y[:-1]
        z= z[:-1]
        q= q[:-1]
    elif np.isnan(x[-1]) or np.isnan(y[-1]) or np.isnan(z[-1]) or np.isnan(q[-1]):
        x = x[:-1]
        y= y[:-1]
        z= z[:-1]
        q= q[:-1]
#     print(np.std(ecg_min),np.std(y_test_min),np.std(y_pred_min),np.median(qual_min))
    
    

In [ ]:
from scipy.stats import pearsonr,spearmanr
q,x,y,z = np.array(q),np.array(x),np.array(y),np.array(z)
for i in np.linspace(0,1,11):
    index = np.where((q>=i)&(q<i+.1))[0]
    if len(index)<2:
        continue
    print(pearsonr(x[index],y[index]),pearsonr(y[index],z[index]),i,len(index))

In [ ]:
# plt.hist(y.reshape(-1),50)
plt.hist(y_pred.reshape(-1),50)
plt.hist(ecg_test.reshape(-1),50)

In [ ]:
np.std(y[0].reshape(1,-1,1),axis=1)

In [ ]:
plt.plot(y_pred1[1].reshape(-1),means_test.reshape(-1),'*')

In [ ]:
from scipy.stats import pearsonr
pearsonr(y_pred1[1].reshape(-1),means_test.reshape(-1))

In [ ]:
import keras

In [ ]:
keras.__version__

In [ ]:
y_pred.reshape(-1).shape

In [ ]:
t = y_pred.reshape(-1)

In [ ]:
t[np.isfinite(t)].shape